In [2]:
import torch

In [3]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from torch.utils.data.dataset import Dataset
from torch.autograd import Variable

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

import os
import pandas as pd
from PIL import Image
from PIL import ImageFilter

Random Seed:  999


In [50]:
class DatasetFromCSV(Dataset):
    def __init__(self, csv_path, transforms=None, pca=None):
        iter = 0
        for path in csv_path:
            if iter == 0:
                self.data = pd.read_csv(path).query("Label<8")
            else:
                self.data = self.data.append(pd.read_csv(path).query("Label<8"))
            iter += 1
        self.data = self.data.reset_index(drop=True)
#        self.data = self.data.query("Label<8")
        self.labels = np.asarray(self.data.iloc[:, 1])
        self.transforms = transforms
        self.pca = pca
        self.imgs = []
        for index in range(self.data.iloc[:, 0].size):
            img_x = Image.open(images_folder + self.data.iloc[:, 0][index])
            img_x = img_x.convert('L')
            img_x = np.array(img_x).reshape(1,-1)[0]
            self.imgs.append(img_x)
        self.imgs = np.array(self.imgs)
        # PCA tranform the images
        self.imgs = pca.transform(self.imgs)
 
    def __getitem__(self, index):
        single_image_label = self.labels[index]
        img = self.imgs[index].reshape(30,30)
        '''
        img = Image.open(images_folder + self.data.iloc[:, 0][index])
        img = img.convert('L')
        # edge detection
        # img = img.filter(ImageFilter.FIND_EDGES)
        '''
        # 将图像转换成 tensor
        if self.transforms is not None:
            img_as_tensor = self.transforms(img)
            # 返回图像及其 label
        
        return (img_as_tensor, single_image_label)
 
    def __len__(self):
        return len(self.data)


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,),(0.5,))])
batch_size = 32



In [4]:
torch.cuda.is_available()

True

In [5]:
train_sets = ["train_subset0.csv", "train_subset1.csv", "train_subset2.csv", "train_subset3.csv", "train_subset4.csv"]
test_sets = ["val_subset0.csv", "val_subset1.csv", "val_subset2.csv", "val_subset3.csv", "val_subset4.csv"]
images_folder = "./images/"
labels_folder = "./labels/"
    

In [6]:
label_dir = os.path.join(os.getcwd(), "labels", train_sets[4])

In [7]:
label__ = pd.read_csv(label_dir).query("Label<8").reset_index(drop=True)
print(label__)

                   Filename  Label
0     20170410-123553-0.jpg      0
1     20171109-072611-1.jpg      5
2     20170714-115750-2.jpg      1
3     20171113-144443-3.jpg      6
4     20171102-120225-2.jpg      2
...                     ...    ...
5039  20171025-082241-2.jpg      3
5040  20171121-084619-1.jpg      6
5041  20170727-140127-1.jpg      4
5042  20170906-113728-1.jpg      3
5043  20170217-095330-0.jpg      7

[5044 rows x 2 columns]


In [8]:
img__ = Image.open(images_folder + label__.iloc[:, 0][1]) 
img__ = img__.convert('L')
#img__.show()
imageWithEdges = img__.filter(ImageFilter.FIND_EDGES)
#imageWithEdges.show()
img__ = np.array(img__)
#img__ = img__.view(-1, 256 * 256)
print( img__ )
print( img__.reshape(1,-1)[0] )

images_samples = []
labels_samples = []

for index in range(200):
    img_x = Image.open(images_folder + label__.iloc[:, 0][index])
    img_x = img_x.convert('L')
    img_x = np.array(img_x).reshape(1,-1)[0]
    images_samples.append(img_x)
    labels_samples.append(label__.iloc[:, 1][index])

images_samples = np.array(images_samples)
print(images_samples.shape)

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


pca = PCA(n_components=150).fit(images_samples)
lda = LinearDiscriminantAnalysis(n_components=7).fit(images_samples, labels_samples)

new_imgs = pca.transform(images_samples)
rev_imgs = pca.inverse_transform(new_imgs)

print(new_imgs.shape)
print(rev_imgs.shape)

print(rev_imgs[0].reshape(256,256).astype('uint8'))

[[ 15  60  71 ...  39  58  62]
 [  5  97  53 ...  83  82  88]
 [ 52  82  25 ... 111  72  66]
 ...
 [ 93 113 120 ... 110 132 116]
 [125 118 102 ... 113 132 156]
 [140 126 109 ...  61  63 110]]
[ 15  60  71 ...  61  63 110]
(200, 65536)
(200, 150)
(200, 65536)
[[244   8  22 ...  24  18  23]
 [255 238  12 ...  30  20  33]
 [252 255  12 ...  21   3  21]
 ...
 [ 13  21  15 ...  15  15  28]
 [255  12  19 ...   4  20  46]
 [  5  19  12 ...   4  31  45]]


In [10]:
rev_img__ = Image.fromarray(rev_imgs[0].reshape(256,256).astype('uint8'))

rev_img__.show()

img_x = Image.open(images_folder + label__.iloc[:, 0][0])
img_x = img_x.convert('L')
img_x.show()

In [48]:
def generatePCA(train_path):
    labels = pd.read_csv(train_path).query("Label<8").reset_index(drop=True)
    train_images = []
    for index in range(labels.iloc[:, 0].size):
        img_x = Image.open(images_folder + label__.iloc[:, 0][index])
        img_x = img_x.convert('L')
        img_x = np.array(img_x).reshape(1,-1)[0]
        train_images.append(img_x)
    
    train_images = np.array(train_images)
    pca = PCA(n_components=900).fit(train_images)
    return pca
    
pca = generatePCA(labels_folder + train_sets[0])

In [51]:
csv_path = [labels_folder+"train_subset0.csv", labels_folder+"train_subset1.csv", 
            labels_folder+"train_subset2.csv", labels_folder+"train_subset3.csv", labels_folder+"train_subset4.csv"]
test_csv_path = [labels_folder+"val_subset0.csv", labels_folder+"val_subset1.csv", 
            labels_folder+"val_subset2.csv", labels_folder+"val_subset3.csv", labels_folder+"val_subset4.csv"]


train_data= DatasetFromCSV([labels_folder + train_sets[0]],transform,pca)
test_data = DatasetFromCSV([labels_folder + test_sets[0]],transform,pca)
 
train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size)
 
img,lab = next(iter(train_loader))
print(img.shape)
print(len(train_data))
print(len(test_data))

torch.Size([32, 1, 30, 30])
tensor([[[ 8.6343e+03, -4.7154e+03, -1.5083e+03, -1.0609e+03, -6.9808e+02,
           2.2038e+01,  1.7968e+03, -1.5644e+03,  3.7659e+02, -2.7346e+02,
          -9.7788e+02,  3.6708e+02, -1.5190e+02, -2.5441e+03, -5.5122e+02,
          -1.3322e+03,  5.0740e+02, -1.4411e+02, -7.9145e+02, -4.6668e+02,
           1.0706e+03,  1.9249e+02,  4.2524e+02,  1.8133e+02,  4.1831e+01,
          -1.1888e+02,  1.1383e+03, -8.7304e+01, -1.7899e+03, -1.6369e+02],
         [ 3.8242e+02,  3.5686e+02,  9.7100e+02,  5.9472e+02,  6.1771e+02,
           1.0808e+03,  4.1833e+02, -2.3994e+02,  2.7257e+02, -2.1170e+03,
          -7.0386e+02,  5.5099e+02, -2.0554e+02,  2.4663e+02,  3.5465e+02,
           6.6649e+02,  2.5353e+01, -7.9874e+02, -3.6337e+02,  2.1260e+02,
          -3.0372e+02, -2.7911e+02,  6.7331e+02, -8.8022e+02,  1.3823e+03,
          -3.7869e+02,  9.6214e+02, -8.1501e+01,  4.6475e+02, -2.3279e+02],
         [ 1.7028e+03,  1.9350e+02,  7.3971e+02, -1.5934e+02,  3.6627e

In [56]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(input_dim, output_dim)
        #self.linear2 = nn.Linear(4096, 256)
        #self.linear3 = nn.Linear(256, output_dim)

    def forward(self, x):
        out = self.linear1(x)
        # out = nn.functional.relu(out)
        #out = self.linear2(out)
        # out = nn.functional.relu(out)
        #out = self.linear3(out)
        # out = nn.functional.relu(out)
        return out

input_dim = 900
output_dim = 8
epochs = 15 * 2
lr_rate = 0.001
_lamda = 1.2

model = LogisticRegression(input_dim, output_dim)
use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
print(use_gpu)
if use_gpu:
    model = model.cuda()

True


In [57]:
# optimization

criterion = nn.CrossEntropyLoss() # 交叉熵损失
optimizer = torch.optim.SGD(model.parameters(), lr = lr_rate)

In [59]:
for epoch in range(int(epochs)):
    print('epoch {}'.format(epoch + 1))
    running_loss = 0.0
    running_acc = 0.0
    for i, (images, labels) in enumerate(train_loader): #利用enumerate取出一个可迭代对象的内容
        images = images.view(-1, 900)
        if use_gpu:
            images = Variable(images).cuda()
            labels = Variable(labels).cuda()
        else:
            images = Variable(images)
            labels = Variable(labels)

        outputs = model(images.float())
        loss = criterion(outputs, labels)
        
        running_loss += loss.item() * labels.size(0)
        _, pred = torch.max(outputs.data, 1)
        num_correct = (pred == labels).sum()
        running_acc += num_correct.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        '''
        if  i % 100 == 0:
            print (model.parameters())
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, epochs, running_loss / (batch_size * (i+1)),
                running_acc / (batch_size * (i+1))))
        '''
    
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_data)), running_acc / (len(train_data))))
    
    # test the model
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(test_loader):
        images = images.view(-1, 900) 
        if use_gpu:
            images = Variable(images).cuda()
            labels = Variable(labels).cuda()
        else:
            images = Variable(images)
            labels = Variable(labels)
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print(correct, total)
    print('accuracy of the model %.2f' % (100 * correct / total))

epoch 1
Finish 1 epoch, Loss: 8357.995620, Acc: 0.396983
tensor(354, device='cuda:0') 1680
accuracy of the model 21.07
epoch 2
Finish 2 epoch, Loss: 8226.928617, Acc: 0.406511
tensor(351, device='cuda:0') 1680
accuracy of the model 20.89
epoch 3
Finish 3 epoch, Loss: 8478.939410, Acc: 0.401350
tensor(341, device='cuda:0') 1680
accuracy of the model 20.30
epoch 4
Finish 4 epoch, Loss: 8308.375313, Acc: 0.410282
tensor(342, device='cuda:0') 1680
accuracy of the model 20.36
epoch 5
Finish 5 epoch, Loss: 8537.386941, Acc: 0.393609
tensor(341, device='cuda:0') 1680
accuracy of the model 20.30
epoch 6
Finish 6 epoch, Loss: 8270.991356, Acc: 0.412267
tensor(320, device='cuda:0') 1680
accuracy of the model 19.05
epoch 7
Finish 7 epoch, Loss: 8418.574986, Acc: 0.401350
tensor(341, device='cuda:0') 1680
accuracy of the model 20.30
epoch 8
Finish 8 epoch, Loss: 8499.276871, Acc: 0.401548
tensor(338, device='cuda:0') 1680
accuracy of the model 20.12
epoch 9
Finish 9 epoch, Loss: 8451.901233, Acc: 

In [ ]:
# test the model
correct = 0
total = 0

for i, (images, labels) in enumerate(test_loader):
    images = images.view(-1, 256 * 256) 
    if use_gpu:
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
    else:
        images = Variable(images)
        labels = Variable(labels)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    print (predicted)
    correct += (predicted == labels).sum()

print(correct, total)
print('accuracy of the model %.2f' % (100 * correct / total))